In [4]:
import os

In [5]:
# Key
BIGQUERY_CREDENTIALS_FILE_PATH = r"/Users/taylorlin/Desktop/GCP_Key/harry_GCS_BigQuery_write_cred.json"
# BIGQUERY_CREDENTIALS_FILE_PATH = r"D:\data_engineer\TIR_group2\TIR101_Group2\secrets\harry_GCS_BigQuery_write_cred.json"
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = BIGQUERY_CREDENTIALS_FILE_PATH

In [6]:
from google.cloud import bigquery
BQ_CLIENT = bigquery.Client()  
# showing project name(level higher than dataset)
BQ_CLIENT.project

'tir101-group2-422603'

In [7]:
# Dataset Creation
def create_dataset(dataset_name:str , location:str="asia-east1",client: bigquery.Client = BQ_CLIENT):
    """Create a dataset"""
    porject_name = client.project
    dataset_id = f"{porject_name}.{dataset_name}"

    dataset = bigquery.Dataset(dataset_id)
    dataset.location = location

    client.create_dataset(dataset)
    print(f"created {dataset_name} in project {porject_name} ,location:{location}")

create_dataset(dataset_name="MRT_GCS_to_BQ_SRC_ODS_DIM")


created MRT_GCS_to_BQ_SRC_ODS_DIM in project tir101-group2-422603 ,location:asia-east1


# SRC creation

In [9]:
def GCS_MRT_static_data_to_BQ_SRC(client: bigquery.Client = BQ_CLIENT) -> None:
    """create external table (GCS to BQ) for MRT raw data(mrt_station)"""
    query_job = client.query(
    """
        CREATE OR REPLACE EXTERNAL TABLE `MRT_GCS_to_BQ_SRC_ODS_DIM.SRC_MRT_static_data`
        (
            `StationUID` STRING,
            `StationID` STRING,
            `StationAddress` STRING,
            `BikeAllowOnHoliday` BOOL,
            `SrcUpdateTime` TIMESTAMP,
            `UpdateTime` TIMESTAMP,
            `VersionID` STRING,
            `LocationCity` STRING,
            `LocationCityCode` STRING,
            `LocationTown` STRING,
            `LocationTownCode` STRING,
            `StationName_Zh_tw` STRING,
            `StationName_En` STRING,
            `StationPosition_PositionLon` FLOAT64,
            `StationPosition_PositionLat` FLOAT64,
            `StationPosition_GeoHash` STRING            
        )
        OPTIONS 
        (
            format = 'CSV',
            uris = ['gs://static_reference/mrt/mrt_station.csv'],
            skip_leading_rows = 1,
            max_bad_records = 1
        )
    """
    )
    query_job.result()
    print("SRC_MRT_static_data is created.")
GCS_MRT_static_data_to_BQ_SRC()

SRC_MRT_static_data is created.


# BQ_SRC_to_ODS

# DDL
CREATE TABLE `mrt_station` (
  `mrt_station_id` int PRIMARY KEY,
  `mrt_route_id` char(2),
  `sequence` char(2),
  `station_name` varchar(20),
  `station_en` varchar(100),
  `station_address` varchar(150),
  `lat` decimal(8,6),
  `lng` decimal(9,6),
  `city_code` char(3),
  `district` varchar(10),
  `bike_allow_on_holiday` bool,
  `create_time` timestamp,
  `update_time` timestamp
);

AS `mrt_route_id`,、`sequence`沒有找到

In [11]:
def BQ_SRC_to_ODS(client: bigquery.Client = BQ_CLIENT) -> None:
    """Transforming the SRC layer of MRT stations into the ODS layer."""
    query_job = client.query(
    """
        CREATE OR REPLACE TABLE `MRT_GCS_to_BQ_SRC_ODS_DIM.ODS_MRT_static_data` AS
        SELECT
        (
            `StationID` AS `mrt_station_id`,
            AS mrt_route_id,
            AS `sequence`
            `StationName_Zh_tw` AS `station_name`,
            `StationName_En` AS `station_en`,
            `StationAddress` AS `station_address`
            `StationPosition_PositionLat` AS `lat`,
            `StationPosition_PositionLon` AS `lng`,
            `LocationCityCode` AS `city_code`,
            `LocationTown` AS `district`,
            `BikeAllowOnHoliday` AS `bike_allow_on_holiday`,
            `UpdateTime` AS `update_time`         
        )
        FROM 
            MRT_GCS_to_BQ_SRC_ODS_DIM.SRC_MRT_static_data
    """
    )
    query_job.result()
    print("ODS_MRT_static_data is created.")
GCS_MRT_static_data_to_BQ_SRC()

SRC_MRT_static_data is created.
